In [13]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim

In [14]:
# Define transforms for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
])

# Define paths to your train and validation data
train_path = "train"
val_path = "val"

# Load train and validation datasets
train_dataset = datasets.ImageFolder(train_path, transform=transform)
val_dataset = datasets.ImageFolder(val_path, transform=transform)

# Define DataLoader to load data in batches
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

# Print dataset details
print("Number of classes:", len(train_dataset.classes))
print("Classes:", train_dataset.classes)
print("Number of training images:", len(train_dataset))
print("Number of validation images:", len(val_dataset))


Number of classes: 2
Classes: ['MEL', 'NV']
Number of training images: 6426
Number of validation images: 1252


In [15]:
folder = ImageFolder('train', transform=transforms.ToTensor())
loader = DataLoader(folder, batch_size=8, shuffle=True)

Xexamples, Yexamples = next(iter(loader))

<h2> Some baseline model

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class SimpleCNN(nn.Module):
    
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [23]:
import torch.optim as optim
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    # Use tqdm for progress bar
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=running_loss / total, accuracy=correct / total)

    # Print epoch statistics
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2%}")


Epoch 1/10: 100%|██████████| 201/201 [02:28<00:00,  1.35batch/s, accuracy=0.662, loss=0.94] 


Epoch [1/10], Loss: 0.9405, Accuracy: 66.20%


Epoch 2/10: 100%|██████████| 201/201 [02:23<00:00,  1.40batch/s, accuracy=0.721, loss=0.538]


Epoch [2/10], Loss: 0.5383, Accuracy: 72.05%


Epoch 3/10: 100%|██████████| 201/201 [02:12<00:00,  1.52batch/s, accuracy=0.72, loss=0.542] 


...

Epoch [8/10], Loss: 0.6942, Accuracy: 49.21%


Epoch 9/10: 100%|██████████| 201/201 [02:31<00:00,  1.33batch/s, accuracy=0.495, loss=0.694]


Epoch [9/10], Loss: 0.6940, Accuracy: 49.46%


Epoch 10/10: 100%|██████████| 201/201 [02:32<00:00,  1.31batch/s, accuracy=0.505, loss=0.694]

Epoch [10/10], Loss: 0.6937, Accuracy: 50.47%
